In [ ]:
%load_ext autoreload
%autoreload 2

# This file contains tests draft for the Field. It is incomplete.
# It contains only tests which were used during development and which are useful.
import os
import sys
from datetime import date
import warnings
import shutil
from py.path import local
import numpy as np
import pandas as pd

sys.path.append('../..') # for running py-script
sys.path.append('../../..') # for running this notebook directly

warnings.filterwarnings('ignore')

from seismiqb import Field
from seismiqb.src.geometry import export

In [ ]:
# Defaults for run directly from this notebook
# SEED = 10
# SAVING_DIR = './'
# DATESTAMP = date.today().strftime("%Y-%m-%d")
# CUBE_SHAPE = (100, 100, 100)
# DROP_EXTRA_FILES = True
# GITHUB_MODE = False

# Preparation

**Storage structure:**
___



**field_test_files** (tests root directory with temporary files)

&emsp;├── **test_array.npy** (Test cube data, now it is a random noise)

&emsp;├── **test_cube.sgy** (Test cube data in the sgy format)

&emsp;├── **test_cube.meta** (Meta data of the test cube)
    
&emsp;├── **test_matrix** (Saved test matrix)

&emsp;└── **test_points** (Saved test points)


In [ ]:
# Storage preparation:
# The test_dir contains temporary files
test_dir = "field_tests_files"

if GITHUB_MODE:
    test_dir_path = SAVING_DIR.mkdir(test_dir)

else:
    # Clear and recreate workspace
    test_dir_path = os.path.join(SAVING_DIR, test_dir)

    try:
        shutil.rmtree(test_dir_path)
    except OSError as e:
        print(f"Can't delete the directory {test_dir_path} : {e.strerror}")

    if not os.path.exists(test_dir_path):
        os.makedirs(test_dir_path)

CUBE_PATH = os.path.join(test_dir_path, f'test_cube_{DATESTAMP}.sgy')

In [ ]:
%%time
# Create a fake cube
rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

with open(os.path.join(test_dir_path, f'test_array_{DATESTAMP}.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False,
                            sample_rate=2., delay=50, pbar='t')

In [ ]:
%%time
# Init a field
field = Field(geometry=CUBE_PATH)

# Tests

In [ ]:
%%time
# Test dump_charisma, load_charisma and is_charisma_like (matrix case)

# Create and dump a random matrix
matrix = rng.integers(low=0, high=10, size=CUBE_SHAPE[:2])

path=os.path.join(test_dir_path, f'test_matrix_{DATESTAMP}')

field.dump_charisma(data=matrix, path=path, format='matrix', name='saved_matrix')

# Check is_charisma_like
assert field.is_charisma_like(path), "A dumped matrix file is not charisma-like."

# Open saved data
opened_matrix = field.load_charisma(path=path, format='matrix')

assert np.array_equal(matrix, opened_matrix), "Saved and loaded matrices are not the same."

In [ ]:
%%time
# Test dump_charisma, load_charisma and is_charisma_like (points case)

# Create random points in the field
i_lines_range = np.linspace(0, CUBE_SHAPE[0]-1, CUBE_SHAPE[0]).astype(int)
x_lines_range = np.linspace(0, CUBE_SHAPE[1]-1, CUBE_SHAPE[1]).astype(int)

i_lines, x_lines = np.meshgrid(i_lines_range, x_lines_range)

depths = rng.integers(low=0, high=10, size=(i_lines.size, 1))

points = np.hstack([i_lines.reshape(i_lines.size, 1), x_lines.reshape(x_lines.size, 1), depths])

# Sort for dumped and loaded points consistency
points = np.sort(points, axis=1)
points = np.sort(points, axis=0)

# Save points as charisma
path=os.path.join(test_dir_path, f'test_points_{DATESTAMP}')

field.dump_charisma(data=points, path=path, format='points', name='saved_points')

# Check is_charisma_like
assert field.is_charisma_like(path), "A dumped points file is not charisma-like."

# Open saved data
opened_points = field.load_charisma(path=path, format='points')

assert np.array_equal(points, opened_points), "Saved and loaded points arrays are not the same."

# Exit

In [ ]:
# Remove extra files
if DROP_EXTRA_FILES and not GITHUB_MODE:
    try:
        shutil.rmtree(test_dir_path)
    except OSError as e:
        print(f"Can't delete the directory {test_dir_path} : {e.strerror}")